In [2]:
from IPython import get_ipython
from IPython.display import display

import openai
import os
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time


In [3]:
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
def parse_user_input(user_input):
    prompt = f"""
Extract structured buying rules from this command.

Command: "{user_input}"

Return output only in JSON format like this:

{{
  "product": "<product name>",
  "platform": "Amazon",
  "target_price": 800.0,
  "min_seller_rating": 4.0
}}

Respond only with valid JSON.
"""


    # Use the new client.chat.completions.create method
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )

    # Access the content from the response object using dot notation
    output = response.choices[0].message.content
    print("🔍 Parsed Rules from OpenAI:\n", output)
    return eval(output)

In [5]:
def get_product_data_amazon(url):
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument(f"--user-data-dir=/tmp/unique-user-dir-{time.time()}")  # ✅ avoids conflict

    driver = None

    try:
        driver = webdriver.Chrome(options=options)
        driver.get(url)

        time.sleep(3)

        soup = BeautifulSoup(driver.page_source, "html.parser")

        try:
            # Select multiple possible price elements
            price_element = soup.select_one("#priceblock_ourprice, #priceblock_dealprice, .a-price-whole")
            if price_element:
                 # Handle different price formats (e.g., the new '.a-price-whole' class)
                price_text = price_element.get_text()
                # Clean price text: remove currency symbols, commas, and potential trailing dots from whole number
                price_text = price_text.replace('$', '').replace(',', '').strip()
                # If the price_element was .a-price-whole, the fractional part might be in .a-price-fraction
                fractional_element = price_element.find_next_sibling('.a-price-fraction')
                if fractional_element:
                    price_text += '.' + fractional_element.get_text().strip()

                # Attempt conversion to float, handle potential errors
                try:
                    price = float(price_text)
                except ValueError:
                     price = None # Handle cases where conversion fails
            else:
                price = None # No price element found

        except Exception as e: # Catch more general exceptions during price scraping
            print(f"Error scraping price: {e}")
            price = None

        try:
            # Select element containing star rating text
            rating_element = soup.select_one("span.a-icon-alt")
            if rating_element:
                rating_text = rating_element.get_text()
                # Extract the first number from the rating text (e.g., "4.5 out of 5 stars")
                try:
                    rating = float(rating_text.split()[0])
                except (ValueError, IndexError):
                    rating = None # Handle cases where rating format is unexpected
            else:
                 rating = None # No rating element found

        except Exception as e: # Catch more general exceptions during rating scraping
            print(f"Error scraping rating: {e}")
            rating = None

        return {"price": price, "rating": rating}
    except Exception as e:
        # Catch the SessionNotCreatedException specifically or other potential errors during driver init/page load
        print(f"An error occurred during Selenium operation: {e}")
        return {"price": None, "rating": None} # Return None for price and rating on error
    finally:
        # Ensure driver is quit only if it was successfully initialized
        if driver is not None:
            driver.quit()

In [6]:
user_command = input("Enter your command (e.g., 'Track iPhone. Buy if under $800'): ")
product_url = input("Enter product URL (Amazon): ")


Enter your command (e.g., 'Track iPhone. Buy if under $800'): Track iPhone. Buy if under $800
Enter product URL (Amazon): https://www.amazon.com/Apple-iPhone-14-128GB-Midnight/dp/B0BN72FYFG/ref=sr_1_1?dib=eyJ2IjoiMSJ9.Y6ERtAXi4bn8Q3Q1ZFAAuaJCwZbxrM1XDk23xQFfDKH_QRY2yrOBKk9WcJU6LseHciS6c4TXX0luShDRZeOdThd4tS9qehMAGlFZv8JCg7MM3rNrDrlBL4jD5D7iPnGoSrI3AHoLiIczD8TLha18EDfTaM13cMn85OrETdTIMNV75C_Z_FKusMFOeYxqtMEDEEB9y0TPsQn_sfv8mMpAEj0-gIhu80-0EkXJtmvVgmU.kQPSCYc2GWpBiQCbExahpwZnE27t6AxJXDAR41TPaMw&dib_tag=se&keywords=iphone&qid=1750348617&sr=8-1&th=1


In [7]:
rules = parse_user_input(user_command)
product_data = get_product_data_amazon(product_url)

print("\n🛍️ Product Data from Amazon:")
print(product_data)

decision = "Do Not Buy"
if product_data["price"] and product_data["rating"]:
    if (product_data["price"] <= rules["target_price"] and
        product_data["rating"] >= rules["min_seller_rating"]):
        decision = "✅ BUY"
    else:
        decision = "❌ DO NOT BUY"

print("\n🤖 Decision:", decision)

🔍 Parsed Rules from OpenAI:
 {
  "product": "iPhone",
  "platform": "Amazon",
  "target_price": 800.0,
  "min_seller_rating": 4.0
}

🛍️ Product Data from Amazon:
{'price': 476.0, 'rating': 4.0}

🤖 Decision: ✅ BUY
